 <img src="index.png" width="240" height="240" align="center"/>
<h1><center> Crime & Economy (in python) </center></h1>

In this project I look for a relationship between economic data and crime rates. 

I obtained statistics for various industries at the county level from the BLS. I also found the number of arrests and charges for a variety of crimes at the county level from the Inter-University Consortium for Political and SOcial Research. These datasets span 2010 to 2016 in two year intervals.

After cleaning the data I will test out some statistical analysis.

\---------------------------------------------------------------------------

BLS, link to data: https://www.bls.gov/cew/datatoc.htm

ICPSR / Crime, link to data: https://www.icpsr.umich.edu/icpsrweb/NACJD/series/57 (seires name: "Uniform Crime Reporting Program Data: County-Level Detailed Arrest and Offense Data, United States, yyyy")

In [203]:
# Project
%run Imports.ipynb
# Imports functions from seperate notebook
%run Functions.ipynb

Notebooky Directory:  C:\Users\Matt\Downloads\Project


Here I load in the data and spit out some summary statistics on the structure of the data.

In [2]:
# Reading in Data
start_time = timeit.default_timer()
# Keys to extract formatted State and County names
key = pd.read_excel('fips_codes_website.xls')   # Counties and major cities
key00 = pd.read_csv('FIPS_countycodes.txt',delimiter=',')  # Counties only
key00['GU Name'] = key00['GU Name'].str[:-7]  # Slice 'county' from names
# Crime
crime2010 = pd.read_csv('33523-0001-crimedata2010.txt',delimiter='\t')
crime2012 = pd.read_csv('35019-0001-crimedata2012.txt',delimiter='\t')
crime2014 = pd.read_csv('36399-0001-crimedata2014.txt',delimiter='\t')
crime2016 = pd.read_csv('37059-0001-crimedata2016.txt',delimiter='\t')
# Enumployment & Wages
#test_wages_18 = pd.read_excel('allhlcn183.xlsx',sheet_name='US_St_Cn_MSA')
wage_unemp2018 = pd.read_csv('allhlcn183wage_unemp2018.txt',delimiter='\t',dtype={'St':str})
wage_unemp2016 = pd.read_csv('allhlcn163wage_unemp2016.txt',delimiter='\t',dtype={'St':str})
wage_unemp2014 = pd.read_csv('allhlcn143wage_unemp2014.txt',delimiter='\t',dtype={'St':str})
wage_unemp2012 = pd.read_csv('allhlcn123wage_unemp2012.txt',delimiter='\t',dtype={'St':str})
wage_unemp2010 = pd.read_csv('allhlcn103wage_unemp2010.txt',delimiter='\t',dtype={'St':str})

print("2014 crime:",crime2014.shape,"// key:",key.shape)
print('2018 Wages:',wage_unemp2018.shape)
x = ((crime2014.shape[0])*(crime2014.shape[1]))*4 + ((wage_unemp2018.shape[0])*(wage_unemp2018.shape[1]))*5 + (key00.shape[0])*key00.shape[1]
print(x,'unique datapoints in all datasets')
elapsed = timeit.default_timer() - start_time
print(round(elapsed,3),'run time')

2014 crime: (3177, 56) // key: (41787, 7)
2018 Wages: (62724, 21)
7313843 unique datapoints in all datasets
7.122 run time


In [85]:
# Data Cleaning #
pd.options.mode.chained_assignment = None # Turn off copy with setting warning
start_time = timeit.default_timer()
# Yearly cleaned databases
crime10 = crime_cleaner(crime2010)
crime12 = crime_cleaner(crime2012)
crime14 = crime_cleaner(crime2014)
crime16 = crime_cleaner(crime2016)
cols = list(crime14.columns.values) # Grab columns
# Cleaned Wage Data
economic10 = wage_cleaner(wage_unemp2010) # Returns county level data sets
economic12 = wage_cleaner(wage_unemp2012)
economic14 = wage_cleaner(wage_unemp2014)
economic16 = wage_cleaner(wage_unemp2016)
economic18 = wage_cleaner(wage_unemp2018)
economic18['Industry'] = economic18['Industry'].str[1:] # Remove first space in 2018 industry codes
# Convert all values to floats
z = economic10.columns.tolist() # List of column names to manipulate
y = z[-8:]
economic0 = bot99(economic10[y])
economic10 = pd.concat([economic10[z[:8]],economic0],1)
economic2 = bot99(economic12[y])
economic12 = pd.concat([economic12[z[:8]],economic2],1)
economic4 = bot99(economic14[y])
economic14 = pd.concat([economic14[z[:8]],economic4],1)
economic6 = bot99(economic16[y])
economic16 = pd.concat([economic16[z[:8]],economic6],1)
economic8 = bot99(economic18[y])
economic18 = pd.concat([economic18[z[:8]],economic8],1)

# Combined crime and economic dataframes and rearrange columns
db2010 = crime10.merge(economic10,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
cols = db2010.columns.tolist()  # Code to rearrange columns
cols = ['Year']+cols[:-14]+cols[-10:]
db2010 = db2010[cols]
db2012 = crime12.merge(economic12,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2012 = db2012[cols]
db2014 = crime14.merge(economic14,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2014 = db2014[cols]
db2016 = crime16.merge(economic16,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
db2016 = db2016[cols]
elapsed = timeit.default_timer() - start_time
print(round(elapsed,3),'run time')

8.327 run time


Here I generate a sample dataset of randomly selected counties and all associated data. 

It is worth noting that this can be computationally expensive and that cell run times take about 0.1 seconds per county in the sample. 

(e.g. a sample of 400 counties will take ~40 seconds to generate in the cell below)

In [86]:
# Selects X Random counties and stores them as a dictionary by count // Key = MI, County
start_time = timeit.default_timer()
samp_db = {}
for n in range(100):
    r = random.randint(0,len(crime12))
    x = crime12['State Abbreviation'].iloc[r] # Pull State Abreviation
    y = crime12['GU Name'].iloc[r]  # Pull analogous county name
    z = x+', '+y # Format name for dictionary key
    agg = test_search(y,x) # Return criminal and economic data
    samp_db[z] = agg  # Add data to empty dictionary
keys = list(samp_db.keys())
print(len(samp_db))
print(samp_db.keys())
elapsed = timeit.default_timer() - start_time
print(round(elapsed,3),'run time')

100
dict_keys(['IN, Tipton', 'MN, Cook', 'ND, Grant', 'TN, Houston', 'IA, Appanoose', 'TX, Hidalgo', 'IN, Washington', 'RI, Washington', 'VA, Hanover', 'VA, Highland', 'PA, Snyder', 'MA, Plymouth', 'AR, Miller', 'SD, Turner', 'KS, Lyon', 'WV, Ritchie', 'MN, Pipestone', 'KS, Clark', 'GA, Candler', 'AL, Jackson', 'TX, Bell', 'ND, Ransom', 'KS, Sherman', 'MN, Pine', 'WV, Cabell', 'GA, Webster', 'NE, Merrick', 'MI, Newaygo', 'AL, Etowah', 'MO, Daviess', 'MI, Manistee', 'IA, Cedar', 'MO, St. Clair', 'OR, Coos', 'TX, Hartley', 'OH, Summit', 'NE, Cass', 'TX, Anderson', 'TN, Pickett', 'MO, New Madrid', 'SD, Roberts', 'ND, Grand Forks', 'TX, Jim Wells', 'KS, Crawford', 'ID, Power', 'IA, Adams', 'KY, Daviess', 'IN, Monroe', 'NE, Fillmore', 'MS, Humphreys', 'PA, Union', 'FL, Pinellas', 'MO, Washington', 'TN, Hickman', 'NE, Garfield', 'GA, Paulding', 'MD, Montgomery', 'MI, Iron', 'OH, Knox', 'MI, Benzie', 'GA, Towns', 'MN, Norman', 'KS, Comanche', 'MO, McDonald', 'NC, Wilson', 'TN, Roane', 'IL, Ro

In [89]:
# Select 5 biggest industries in sample
start_time = timeit.default_timer()
industry = big_industry_bot(samp_db)
print('5 Largest Industries in sample:\n',industry)
elapsed = timeit.default_timer() - start_time
print(round(elapsed,3),'run time')

5 Largest Industries in sample:
 ['Goods-producing', 'Natural resources and mining', 'Service-providing', 'Trade, transportation, and utilities', 'Other services']
0.027 run time


In [183]:
# start base data frame -- create new dataframe, still need to account for various industries


bedrock = samp_db # Initialize sample to new variable for trouble shooting // preserve OG sample data

new_world = pd.DataFrame()

for a in keys:
    for b in industry[:]:
        x = bedrock[a].loc[bedrock[a]['Industry'] == b] #
        new_world = new_world.append(x)
        
new_world = new_world.reset_index()  # reset index
new_world = new_world.drop('index',axis=1)  # drop index column
cols = new_world.columns.tolist() # rearrange columns
cols = cols[:3]+cols[-9:]+cols[3:48]
new_world = new_world[cols]

elapsed = timeit.default_timer() - start_time
print(round(elapsed,3),'run time')
new_world.head(10)

4.063 run time


Year State Abbreviation GU Name                      Industry  \
0  2010                 IN  Tipton               Goods-producing   
1  2012                 IN  Tipton               Goods-producing   
2  2014                 IN  Tipton               Goods-producing   
3  2016                 IN  Tipton               Goods-producing   
4  2010                 IN  Tipton  Natural resources and mining   
5  2012                 IN  Tipton  Natural resources and mining   
6  2014                 IN  Tipton  Natural resources and mining   
7  2016                 IN  Tipton  Natural resources and mining   
8  2010                 IN  Tipton             Service-providing   
9  2012                 IN  Tipton             Service-providing   

   Establishment Count  July Employment  August Employment  \
0                   85           1949.0             1519.0   
1                   84           2063.0             1546.0   
2                   95           1783.0             1782.0   
3                   91           1882.0             1861.0   
4                   21            944.0              465.0   
5                   23            987.0              408.0   
6                   26            393.0              395.0   
7                   28            394.0              390.0   
8                  201           1935.0             1851.0   
9                  194           1996.0             1889.0   

   September Employment  Total Quarterly Wages  Average Weekly Wage   ...     \
0                  1586             15259529.0                  697   ...      
1                  1646             16059537.0                  705   ...      
2                  1806             21018542.0                  903   ...      
3                  1931             24884636.0                 1012   ...      
4                   520              5195414.0                  622   ...      
5                   506              5007097.0                  608   ...      
6                   417              4373145.0                  837   ...      
7                   414              4459916.0                  859   ...      
8                  1767             15168739.0                  630   ...      
9                  1877             15057002.0                  603   ...      

   OFAGFAM  DUI  LIQUOR  DRUNK  DISORDR  VAGRANT  ALLOTHR  SUSPICN  CURFEW  \
0       20   46      36     21        6        0       98        0      13   
1       39   33      11     37        1        0       64        0       2   
2        3   41       7     11        0        0       64        0       8   
3       15   61       9      7        3        0      114        0       2   
4       20   46      36     21        6        0       98        0      13   
5       39   33      11     37        1        0       64        0       2   
6        3   41       7     11        0        0       64        0       8   
7       15   61       9      7        3        0      114        0       2   
8       20   46      36     21        6        0       98        0      13   
9       39   33      11     37        1        0       64        0       2   

   RUNAWAY  
0        1  
1        3  
2        2  
3        8  
4        1  
5        3  
6        2  
7        8  
8        1  
9        3  

[10 rows x 57 columns]

Due to inconsistencies in the data it is important that we double check that there is the same number of data entries for each industry we want to observe. If we have an uneven database, we need to draw a new sample. 

In [184]:
# Check counts of target data to ensure symmetry/integrity for analysis
start_time = timeit.default_timer()
crimes = cols[12:]
bot98(new_world,industry)  # Call function to check data
elapsed = timeit.default_timer() - start_time
print(round(elapsed,3),'run time')

Count of target industries in sample 

 MUST ALL BE SAME

Check
Check
Check
Check
0.002 run time


In order to account for changes in population size, we must convert all of our data into percentage changes from a base year(2010). This transformation will cause us to lose a year of data entries but will provide a more robust model. 

In [257]:
# Convert dataframe into percent changes
start_time = timeit.default_timer()
z = new_world.columns.tolist()
tail = 0
head = 4
# .iloc using loop to apply percent change for every 4 years
for a in range(int(len(new_world)/4)):
    new_world.iloc[tail:head] = pd.concat([new_world.iloc[tail:head][z[:4]],new_world.iloc[tail:head][z[4:]].pct_change()],1)
    head += 4
    tail += 4
print(round(elapsed,3),'run time')

23.828 run time


In [256]:
# Create final data frame that accounts for metrics across industries
start_time = timeit.default_timer()
samp_dic = {}
stat = ['Establishment Count','Average Weekly Wage','August Employment']

for s in stat:
    for n in industry:
        samp_dic[n+' '+s] = test_func(new_world,keys,s,n)

for a in crimes:
    samp_dic[a] = test_funk(new_world,a)
    
finaldb = pd.DataFrame(data=samp_dic)

print(samp_dic.keys(),'\n\n')
elapsed = timeit.default_timer() - start_time
# Remove NAs and infinite values
cols = finaldb.columns.tolist()
finaldb[cols[15:]] = finaldb[cols[15:]].fillna(0) # Fill all NaNs with 0's in crime data(because cannot divide by zero)
finaldb = finaldb.dropna(how='any').replace(-np.inf,-0.99).replace(np.inf,0.99)  # drop all 2010/base year data
print(round(elapsed,3),'run time')
finaldb.head(10)

dict_keys(['Goods-producing Establishment Count', 'Natural resources and mining Establishment Count', 'Service-providing Establishment Count', 'Trade, transportation, and utilities Establishment Count', 'Other services Establishment Count', 'Goods-producing Average Weekly Wage', 'Natural resources and mining Average Weekly Wage', 'Service-providing Average Weekly Wage', 'Trade, transportation, and utilities Average Weekly Wage', 'Other services Average Weekly Wage', 'Goods-producing August Employment', 'Natural resources and mining August Employment', 'Service-providing August Employment', 'Trade, transportation, and utilities August Employment', 'Other services August Employment', 'MURDER', 'P1TOT', 'P1VLNT', 'P1PRPTY', 'RAPE', 'ROBBERY', 'AGASSLT', 'BURGLRY', 'LARCENY', 'MVTHEFT', 'ARSON', 'OTHASLT', 'FRGYCNT', 'FRAUD', 'EMBEZL', 'STLNPRP', 'VANDLSM', 'WEAPONS', 'COMVICE', 'SEXOFF', 'DRUGTOT', 'DRGSALE', 'COCSALE', 'MJSALE', 'SYNSALE', 'OTHSALE', 'DRGPOSS', 'COCPOSS', 'MJPOSS', 'SYNP

Goods-producing Establishment Count  \
1                             -0.011765   
2                              0.130952   
3                             -0.042105   
11                            -0.105263   
17                            -0.043478   
18                             0.022727   
19                            -0.022222   
21                            -0.036075   
22                             0.005240   
23                             0.008191   

    Natural resources and mining Establishment Count  \
1                                           0.095238   
2                                           0.130435   
3                                           0.076923   
11                                         -0.142857   
17                                          0.000000   
18                                          0.000000   
19                                          0.250000   
21                                         -0.005000   
22                                         -0.015075   
23                                         -0.045918   

    Service-providing Establishment Count  \
1                               -0.034826   
2                               -0.030928   
3                                0.010638   
11                              -0.074074   
17                              -0.073955   
18                               0.031250   
19                              -0.037037   
21                               0.073306   
22                               0.036081   
23                               0.034334   

    Trade, transportation, and utilities Establishment Count  \
1                                            0.000000          
2                                           -0.068493          
3                                           -0.102941          
11                                          -0.038462          
17                                          -0.101695          
18                                           0.000000          
19                                          -0.075472          
21                                           0.089895          
22                                           0.037730          
23                                           0.010147          

    Other services Establishment Count  Goods-producing Average Weekly Wage  \
1                            -0.090909                             0.011478   
2                             0.000000                             0.280851   
3                             0.200000                             0.120709   
11                           -0.250000                            -0.011534   
17                           -0.032258                             0.078431   
18                            0.066667                             0.062338   
19                           -0.093750                             0.200489   
21                           -0.019663                             0.158147   
22                           -0.005731                             0.093793   
23                           -0.038905                            -0.007566   

    Natural resources and mining Average Weekly Wage  \
1                                          -0.022508   
2                                           0.376645   
3                                           0.026284   
11                                          0.109510   
17                                          0.990000   
18                                          0.328629   
19                                          0.127466   
21                                          0.237265   
22                                          0.138678   
23                                         -0.108468   

    Service-providing Average Weekly Wage  \
1                               -0.042857   
2                                0.024876   
3                                0.076052   
11                               0.085950   


# Statistical Analysis

In [265]:
corrdb = finaldb.corr().abs().sum()
corrdbl = pd.DataFrame(data=corrdb).nlargest(11,0).transpose()  # Largest correlations
corrdbs = pd.DataFrame(data=corrdb).nsmallest(10,0).transpose()   # Smallest correlations
# Creates a correlation matrix of all variables
plt.matshow(finaldb.corr())
plt.suptitle('Correlation Matrix Of All Variables\n')
plt.show()
cols = finaldb.columns.tolist()
print('\n11 HIGHEST total correlations:')
corrdbl


11 HIGHEST total correlations:


P1TOT    DRUGTOT   P1PRPTY  LARCENY   BURGLRY    P1VLNT   OFAGFAM  \
0  10.62317  10.114668  9.720693  9.05804  8.814486  8.794006  8.630524   

   AGASSLT   STLNPRP  Service-providing August Employment   OTHASLT  
0  8.58431  7.884307                             7.818889  7.644763

In [262]:
l = corrdbl.columns.tolist() # variables to regress on
exp = l[1:] # Selects most variable with highest total correlation to use as dependant variable
dep = l[0] # Uses next 10 most correlated variables to act as independant variables

# REGRESSION

Regression is of most correlated variable on next 10 most correlated variables

In [263]:
x = finaldb[exp]  # Select columns to use as independant vars
y = finaldb[dep]  # Set y variable

model = sm.OLS(y, x.astype(float)).fit()
predictions = model.predict(x) # make the predictions by the model

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  P1TOT   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                     231.9
Date:                Tue, 04 Jun 2019   Prob (F-statistic):          1.21e-120
Time:                        15:11:23   Log-Likelihood:                 73.520
No. Observations:                 261   AIC:                            -127.0
Df Residuals:                     251   BIC:                            -91.40
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
DR

# Regression of most correlated variable on all economic variables

In [264]:
x = finaldb[finaldb.columns.tolist()[:15]]  # Select columns to use as independant vars
y = finaldb[dep]  # Set y variable

model = sm.OLS(y, x.astype(float)).fit()
predictions = model.predict(x) # make the predictions by the model

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  P1TOT   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     9.288
Date:                Tue, 04 Jun 2019   Prob (F-statistic):           3.93e-17
Time:                        15:22:34   Log-Likelihood:                -171.49
No. Observations:                 261   AIC:                             373.0
Df Residuals:                     246   BIC:                             426.4
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------

# Regression Summary

The first regression likely has the highest R-squared in our sample. Perhaps unsurprisingly we see that crime rates are more related to each other than with or between economic data. We also see that aggregate variables tend to be the strongest which makes sense again due to their interconnected nature.




In the second regression we can see the most correlated crime rate (P1TOT: total crimes) regressed on all of our economic variables. It is a substantially lower R-squared value, however it is still significant (~35%). 

**In summary we see that changes in establishment counts, average weekly wages, and employment numbers for the 5 largest industries explain about 35% of the changes in total crimes.** 